<a href="https://colab.research.google.com/github/tayhym/10-601-ML/blob/master/python_session_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Python Tutorial Session 2 for IRL (24/04/2020)
Written By: John Lee

## Objective
- Gain some hands-on experience with Python
- Find out more about some of the useful Python libraries

## Outline
1. Review
2. Example: Signal Processing - Backprojection


## 1. Review
Last time, we saw that there are some differences between MATLAB and Python. In the following few cells, we will revise our understanding of the differences by doing a few examples, translating some MATLAB stuff to Python.

In [0]:
import numpy as np

In [0]:
# TODO: Convert the following MATLAB code to Python
# MATLAB: x_axis = -50:0.25:50;
# Hint: Run '?np.arange' for help on the numpy function arange


In [0]:
# TODO: Let's look at the first two elements of x_axis. They should be [-50, -49.75]
# MATLAB: x_axis(1:2)


In [0]:
# TODO: Let's look at the last two elements of x_axis. They should be [49.75, 50]
# MATLAB: x_axis(end-1:end)


Let's take a look at `x_axis` by plotting it. In MATLAB, we would do something like
```
figure();
plot(x_axis);
```
The Python equivalent is not too different! The only difference is we need to import the plotting library (`matplotlib.pyplot`). The vast majority of people import it as `plt` (just like how `numpy` is usually imported as `np`). All we need to do now is add `plt` before each of the MATLAB commands!

In [0]:
import matplotlib.pyplot as plt
plt.plot(x_axis)
plt.show()

## 2. Example: Signal Processing - GBP
In the following example, we shall look at how python can be used for signal processing. To that end, we shall implement global backprojection in Python. 

### Step 1: Downloading Data
For this example, we shall use the AFRL GOTCHA Volumetric SAR Dataset. A small subset of the data shall be loaded from my google Drive folder. The following cell should print the following code block (with the exception of the timestamp)
```
-rw-r--r-- 1 root root 394K Apr 21 03:31 data_3dsar_pass1_az001_HH.mat
-rw-r--r-- 1 root root 394K Apr 21 03:31 data_3dsar_pass1_az002_HH.mat
-rw-r--r-- 1 root root 398K Apr 21 03:31 data_3dsar_pass1_az003_HH.mat
```


In [0]:
!wget -q --load-cookies /tmp/cookies.txt "https://docs.google.com/uc?export=download&confirm=$(wget --quiet --save-cookies /tmp/cookies.txt --keep-session-cookies --no-check-certificate 'https://docs.google.com/uc?export=download&id=1V2Wm20kMfT4EVx9lAcW-IjylXQKiT9Ut' -O- | sed -rn 's/.*confirm=([0-9A-Za-z_]+).*/\1\n/p')&id=1V2Wm20kMfT4EVx9lAcW-IjylXQKiT9Ut" -O data_3dsar_pass1_az001_HH.mat && rm -rf /tmp/cookies.txt
!wget -q --load-cookies /tmp/cookies.txt "https://docs.google.com/uc?export=download&confirm=$(wget --quiet --save-cookies /tmp/cookies.txt --keep-session-cookies --no-check-certificate 'https://docs.google.com/uc?export=download&id=1CJTABx01kvs5pSt6nWCf9sdAjLv-rg-T' -O- | sed -rn 's/.*confirm=([0-9A-Za-z_]+).*/\1\n/p')&id=1CJTABx01kvs5pSt6nWCf9sdAjLv-rg-T" -O data_3dsar_pass1_az002_HH.mat && rm -rf /tmp/cookies.txt
!wget -q --load-cookies /tmp/cookies.txt "https://docs.google.com/uc?export=download&confirm=$(wget --quiet --save-cookies /tmp/cookies.txt --keep-session-cookies --no-check-certificate 'https://docs.google.com/uc?export=download&id=1HNLc7Tw159J1PdIO1ULxLEiSL6EaaLMa' -O- | sed -rn 's/.*confirm=([0-9A-Za-z_]+).*/\1\n/p')&id=1HNLc7Tw159J1PdIO1ULxLEiSL6EaaLMa" -O data_3dsar_pass1_az003_HH.mat && rm -rf /tmp/cookies.txt
!ls -lh | grep data_3dsar_pass1

### Step 2: Reading the data
The GOTCHA dataset is stored in `*.mat` files. That's a MATLAB format, how do we read it with Python? Enter `scipy` (pronounced "sigh-pie"). `scipy` is a Python library that has a lot of functions that are useful for scientific computation (e.g. FFT, windowing functions, filter design), as well as a way to read data from other popular scientific computation tools.

In order to use the `scipy` library, we would first have to import it.

In [0]:
import scipy.io as sio
from scipy.constants import speed_of_light

So far we've mainly imported entire libraries, giving us access to all functions within the library. But sometimes we only need one thing. For example, we only need the `speed_of_light` (and we don't need other constants like the Boltzmann's constant), so we can choose to only import that single variable. The way we do that is by specifying which library we want to take `from`, and subsequently `import` the variable or function that we need.

Next up, we load the MATLAB file.

In [0]:
mat_data = sio.loadmat('./data_3dsar_pass1_az001_HH.mat')["data"][0][0]

Within each `*.mat` file is a MATLAB struct called data, containing the follow fields:
1. `fp` - RADAR data in the frequency domain, after matched-filter.
2. `freq` - The corresponding frequency points
3. `x` - The antenna's x coordinate (in some local coordinate system with the origin at the scene centre)
4. `y` - The antenna's y coordinate (in some local coordinate system with the origin at the scene centre)
5. `z` - The antenna's z coordinate (in some local coordinate system with the origin at the scene centre)
6. `r0` - The range from the antenna to the scene centre. This together with `th` and `phi` locate the antenna in a spherical coordinate system. This should be equal to the distance from (x, y, z) to the origin, but somehow there are some small deviations. I'm not sure what's the deal with this.
7. `th` - The azimuth angle
8. `phi` - The elevation angle
9. `af` - A MATLAB struct containing some auto-focus parameters.

Next, we will extract the components that we will use.

In [0]:
raw_data = mat_data[0]
freq = mat_data[1]
ant_x = mat_data[2]
ant_y = mat_data[3]
ant_z = mat_data[4]

Let's take a look at the frequency axis!

In [0]:
plt.figure()
plt.plot(freq)
plt.show()

Now that we've loaded that, we want to load all the other *.mat files too. We could certainly do something like this:
```python
mat_data = sio.loadmat('./data_3dsar_pass1_az002_HH.mat')["data"][0][0]
raw_data = np.append(raw_data, mat_data[0], axis=1)
ant_x = np.append(ant_x, mat_data[2], axis=1)
ant_y = np.append(ant_y, mat_data[3], axis=1)
ant_z = np.append(ant_z, mat_data[4], axis=1)

mat_data = sio.loadmat('./data_3dsar_pass1_az003_HH.mat')["data"][0][0]
raw_data = np.append(raw_data, mat_data[0], axis=1)
ant_x = np.append(ant_x, mat_data[2], axis=1)
ant_y = np.append(ant_y, mat_data[3], axis=1)
ant_z = np.append(ant_z, mat_data[4], axis=1)
```
But thats..

<figure>
<center>
<img src='https://media.giphy.com/media/10FHR5A4cXqVrO/giphy.gif' />
</center>
</figure>

What do we usually do when the same piece of code is run multiple times with slightly different input parameters? Functions!

In MATLAB, we might have done something like
```matlab
function [raw_data, freq, ant_x, ant_y, ant_z] = extract_gotcha(file_path)
   % do something
   end
```

In Python, we define functions using the keyword `def`, let's try that out by wrapping the following sample code in a function
```python
mat_data = sio.loadmat('./data_3dsar_pass1_az001_HH.mat')["data"][0][0]
raw_data = mat_data[0]
freq = mat_data[1]
ant_x = mat_data[2]
ant_y = mat_data[3]
ant_z = mat_data[4]
```

In [0]:
# TODO: Complete the extract_gotcha script so that it takes in a string containing 
#       the path to the file, and returns raw_data, freq, ant_x, ant_y, ant_z
def extract_gotcha(file_path):
  # Do something
  return raw_data, freq, ant_x, ant_y, ant_z

Let's test out the function, and we'll plot the frequency axis again just to make sure that we've read everything correctly.

In [0]:
(raw_data, freq, ant_x, ant_y, ant_z) = extract_gotcha('./data_3dsar_pass1_az001_HH.mat')
plt.figure()
plt.plot(freq)
plt.show()

Great! Now all we need to do is loop through the *.mat files! In MATLAB, we would have written a for loop like this:
```
for idx = 1:3
  file_str = file_str_array(idx);
  [raw_data, freq, ant_x, ant_y, ant_z] = extract_gotcha(file_str);
end
```
A similar way to do it in python would be:
```python
for idx in range(3): # we can think of range(3) as forming [0, 1, 2]
  # do stuff here
```
We are not limited to looping through numbers in an array. The following cell shows an example where we rotate through filenames.

In [0]:
from os import listdir
files = listdir()
mat_files_seen = 0
for file in files:
  if file.endswith('.mat'):
    if mat_files_seen == 0:
      (raw_data, freq, ant_x, ant_y, ant_z) = extract_gotcha(file)
    else:
      (part_raw_data, part_freq, part_ant_x, part_ant_y, part_ant_z) = extract_gotcha(file)
      raw_data = np.append(raw_data, part_raw_data, axis=1)
      ant_x = np.append(ant_x, part_ant_x, axis=1)
      ant_y = np.append(ant_y, part_ant_y, axis=1)
      ant_z = np.append(ant_z, part_ant_z, axis=1)
    mat_files_seen += 1

Let's check the `shape` (MATLAB `size`) of the loaded variables, and define some helpful parameters (`n_p` and `n_s`)

In [0]:
# TODO: Check the shape of raw_data, freq, ant_x, ant_y, ant_z


In [0]:
# TODO: Define the following helpful parameters
#   n_p - The number of pulses
#   n_s - The number of samples per pulse


Let's concatenate the various components of the antenna position together.

In [0]:
ant_xyz = np.concatenate((ant_x, ant_y, ant_z), axis=0)
ant_xyz.dtype

Oh dear! What is data type of `<f4`? That's a 32-bit little-endian float. Hmm, we want it to be a double, so let's convert it.

In [0]:
ant_xyz = np.double(ant_xyz)
ant_xyz.dtype

In [0]:
wavelength = speed_of_light/np.mean(freq)

### Step 3: Setting Up the Image Grid
Next, we'll start be setting up the image grid. We already started to do this during the review - `x_axis` is the x_axis of the image grid. Next, we'll set up `y_axis`, do a meshgrid and then we should be good to go!

In [0]:
# TODO: Define the y_axis
# MATLAB: y_axis = -50:0.25:50;


In [0]:
?np.meshgrid

In [0]:
# TODO: Convert 1-d x_axis and y_axis to the 2-d x_grid and y_grid


Let's take a look at `x_grid` and `y_grid`. In MATLAB, we would have used something like `imagesc(x_grid)` to display it. What is the equivalent in Python?

`plt.imshow(x_grid)`

In [0]:
plt.figure()
plt.imshow(x_grid)
plt.show()

MATLAB also has the subplot command that allows us to plot multiple things in the same figure. Is there a Python equivalent? Yes!
`plt.subplot()`

In the following cell, display `x_grid` and `y_grid` side-by-side using a single figure with subplots.

*Bonus:*
1. Label your subplots with `plt.title('Your Title Here')`
2. Make the figure larger by specifying the figure size when creating the figure. You can do this with `plt.figure(figsize=(12, 6))`

In [0]:
# TODO: 


We've created our x-axis and our y-axis! One last dimension to go and then we're done with our image grid. The z-axis is all going to be zero. In MATLAB, we would do something like:
`z_grid = zeros(size(x_grid))`

It's not too different in Python. The equivalent `zeros` function is `np.zeros`

In [0]:
# TODO: 


### Step 4: Preparing the data
Now that the imaging grid has been set up, we will prepare the data to be backprojected. The `raw_data` is in the frequency domain. We want an upsampled time-domain version of the signal for subsequent backprojection. Let's first start with getting the axis right.

In [0]:
upsamp_fac = 10
zero_pad = np.zeros((np.int((upsamp_fac-1)/2*n_s), n_p))
freq_spacing = np.mean(np.diff(freq, axis=0))
freq_left_pad = np.arange(-zero_pad.shape[0], 0)*freq_spacing + freq[0]
freq_right_pad = np.arange(1, zero_pad.shape[0] + 1)*freq_spacing + freq[-1]
freq_us = np.concatenate((freq_left_pad, np.squeeze(freq), freq_right_pad), axis = 0)

Great! Now let's plot the upsampled frequency axis, and let's also plot the difference between consecutive values (just to make sure we don't have an off-by-one error).

In [0]:
# TODO: Create a figure with two subplots:
#   Subplot 1: Plot the upsampled frequency axis
#   Subplot 2: Plot the difference between consecutive values
#     Hint for subplot 2: ?np.diff


Now we'll also add the zero pad to the top and bottom of `raw_data`.

In [0]:
raw_data_us = np.concatenate((zero_pad, raw_data, zero_pad), axis=0)

We're going to do an FFT (along with all the fftshifts necessary) to get from the frequency domain to the time domain.

In [0]:
rg_comp = np.fft.fftshift(np.fft.ifft(np.fft.ifftshift(raw_data_us, axes=(0)), axis=0), axes=(0))

Now to form the range axis, there's this nifty python function (for which I don't think there's a MATLAB equivalent). `np.fft.fftfreq(l, ts)` produces the axis after data of length `l` and sampling period `ts` has been put through the fft.

In [0]:
r_axis = np.fft.fftshift(np.fft.fftfreq(freq_us.shape[0], freq_spacing))*speed_of_light/2

In [0]:
plt.figure()
plt.plot(r_axis)
plt.xlabel('Index')
plt.ylabel('Range (m)')
plt.show()

### Step 5: BP


In [0]:
n_row = x_grid.shape[0]
n_col = x_grid.shape[1]

Just like in MATLAB, vectorized operations are a lot faster than nested for loops. So we're going to vectorize our image grid:

In [0]:
grid = np.stack((x_grid, y_grid, z_grid), axis=-1)
# Vectorize Image Grid
grid_vec = np.reshape(grid, (-1, 3))

In [0]:
img = 1j*np.zeros((grid_vec.shape[0], ))
r0 = np.linalg.norm(ant_xyz, axis=0)
for pulse_num in range(n_p):
  # TODO: Select the pulse from rg_comp
  pulse = 
  # TODO: Calculate the distance that the pixel of interest is further from the scene centre
  dist = 
  # TODO: Get the interpolated value query at dist
  interp_val = 
  img += interp_val * np.exp(1j*4*np.pi*dist/wavelength)

In [0]:
img = np.reshape(img, (n_row, n_col))

In [0]:
log_img = 20*np.log10(np.abs(img))
max_val = np.max(np.max(log_img))
plt.figure(figsize=(12,12))
plt.imshow(log_img, cmap='gray', vmin=max_val-60, vmax=max_val, origin='lower', extent=(-50, 50, -50, 50))
plt.show()

Alright! We did it!

<figure>
<center>
<img src='https://media.giphy.com/media/YTbZzCkRQCEJa/giphy.gif' />
</center>
</figure>

---

Bonus (Take-home problem): 
- Apply weighting
  - Option 1: Apply weighting to the raw phase history data
  - Option 2: Re-align phase history after BP then apply weighting
    - Apply location dependent phase compensation to re-align phase history
    \begin{align}
      e^{j\left(\frac{4\pi\gamma}{c^2}d^2 - \frac{4\pi}{\lambda}d\right)}
    \end{align}



## 3. Consolidation

What have we done today?
- Gotten our hands dirty with Python
- Seen different ways to import various libraries
  - `import library as lib`
  - `from library import specific_function`
- Figured out how to plot
  - `import matplotlib.pyplot as plt`
  - Prepend `plt` for most MATLAB plotting functions (e.g. `figure`, `plot`)
  - `imagesc` is `plt.imshow` in Python
- Seen how functions and loops are written in Python
  - Functions defined by 
  ```python
  def function_name(input_param1, input_param2):
      # function code
      return output_param
  ```
  - For Loops:
  ```python
  for idx in range(9):
      if idx+1 % 3 == 0:
        print('Hooray!')
      else:
        print('Hip')
  ```
- Explored some useful libraries for signal processing
  - `numpy`
  - `scipy`
  - `matplotlib`

